# Mapping Groundwater Contaminants of California

Potential goals of this notebook:
1. Clean the dataset into a workable dataframe
2. Spatially plot the data using geopandas, or cartopy
3. ...

## 1. Reorganizing the data

This first section will deal with cleaning and reorganizing the data. The `%matplotlib inline` syntax prints out the figures that are created after each specific call. 

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import os
pd.set_option('display.max_columns', 500)
directory = os.path.abspath('')


The data we are using for this analysis are coming from United States Geological Survey ((USGS)[https://www.usgs.gov/]) observations in the state of California (CA). Specifically, the data was accumulated from the (USGS Water Quality Portal)[https://www.waterqualitydata.us/coverage/]. 

To read in the datafiles, we must make the proper call toward their storage location (on Hydroshare). The following `pd.read_csv` commands may present with some warnings after running. In this instance, the warnings are fine to ignore (however, always be mindful of the coding issues). 


In [4]:
# Enter state code in 'state' variable to read in that states data results.
state = 'CA'
results = pd.read_csv(r'{}/data/{}_result.csv'.format(directory, state))
stations = pd.read_csv(r'{}/data/{}_station.csv'.format(directory, state))

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9,10,11,12,13,14,16,18,19,23,33,37,39,42,47,57,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
results.drop(columns=['OrganizationIdentifier', 
                      'OrganizationFormalName',
                      'ActivityIdentifier', 
                      'ActivityTypeCode', 
                      'ActivityMediaName',
                      'ActivityMediaSubdivisionName',
                      'ResultStatusIdentifier', 
                      'StatisticalBaseCode', 
                      'ResultValueTypeName',
                      'ResultWeightBasisText', 
                      'ResultTimeBasisText',
                      'ResultTemperatureBasisText',
                      'ResultParticleSizeBasisText',
                      'PrecisionValue', 
                      'ResultCommentText',
                      'USGSPCode',
                      'ResultDepthHeightMeasure/MeasureValue',
                      'ResultDepthHeightMeasure/MeasureUnitCode',
                      'ResultDepthAltitudeReferencePointText', 
                      'SubjectTaxonomicName',
                      'SampleTissueAnatomyName',
                      'ResultAnalyticalMethod/MethodIdentifier',
                      'ResultAnalyticalMethod/MethodIdentifierContext',
                      'ResultAnalyticalMethod/MethodName', 
                      'MethodDescriptionText',
                      'LaboratoryName',
                      'AnalysisStartDate', 
                      'ResultLaboratoryCommentText',
                      'DetectionQuantitationLimitTypeName',
                      'DetectionQuantitationLimitMeasure/MeasureValue',
                      'DetectionQuantitationLimitMeasure/MeasureUnitCode',
                      'PreparationStartDate', 
                      'ProviderName',
                      'ProjectIdentifier',
                      'ActivityConductingOrganizationText',
                      'ActivityCommentText',
                      'MeasureQualifierCode', 
                      'SampleCollectionMethod/MethodIdentifier',
                      'SampleCollectionMethod/MethodIdentifierContext',
                      'SampleCollectionMethod/MethodName',
                      'SampleCollectionEquipmentName',
                      'ResultDetectionConditionText'
                     ], inplace=True)